# Installing MASE (again)

Run the block below to install MASE in the current Colab runtime

In [1]:
# git_token = "YOUR_GIT_TOKEN"
# short_code = "YOUR_SHORT_CODE"

# # Check the current python version (It should be using Python 3.10) and update pip to the latest version.
# !python --version
# !python -m pip install --user --upgrade pip

# # Clone MASE from your branch (the branch must already exist)
# !git clone -b lab1_{short_code} https://{git_token}@github.com/DeepWok/mase.git

# # Install requirements
# !python -m pip install -r ./mase/machop/requirements.txt

# # Change working directory to machop
# %cd ./mase/machop/

In [2]:
# !./ch --help

# General introduction

In this lab, you will learn how to use the software stack of MASE. There are in total 7 tasks you would need to finish, and 1 optional task.

# Turning you network to a graph

One specific feature of MASE is its capability to transform DL models to a computation graph using the [torch.fx](<https://pytorch.org/docs/stable/fx.html>) framework.


## Use the Transform functionality without CLI

This tutorial describes how to use the MASE transform functionality for a pre-trained model.

## Import related packages and machop

In [3]:
import sys
import logging
import os
from pathlib import Path
from pprint import pprint as pp

In [4]:
sys.path.append(str(Path.cwd().parent.parent.joinpath("machop")))

from chop.dataset import MaseDataModule, get_dataset_info
from chop.tools.logger import set_logging_verbosity

from chop.passes.graph import (
    save_node_meta_param_interface_pass,
    report_node_meta_param_analysis_pass,
    profile_statistics_analysis_pass,
    add_common_metadata_analysis_pass,
    init_metadata_analysis_pass,
    add_software_metadata_analysis_pass,
)
from chop.tools.get_input import InputGenerator
from chop.tools.checkpoint_load import load_model
from chop.ir import MaseGraph

from chop.models import get_model_info, get_model

set_logging_verbosity("info")

/home/derek/miniconda3/envs/mase/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
INFO     Set logging level to info


## Set up the dataset

Here we create a `MaseDataModule` using the `jsc` dataset from lab1. Note the `MaseDataModule` also requires the name of the model you plan to use data module with. In this case it is `jsc-tiny`.

In [5]:
batch_size = 8
model_name = "jsc-tiny"
dataset_name = "jsc"

# This declares all the params of MaseDataModule but doesn't actually get the datasets
data_module = MaseDataModule(
    name=dataset_name,
    batch_size=batch_size,
    model_name=model_name,
    num_workers=0,
)

# prepare_data is the function which actually gets the underlying train, test, val, etc.. datasets
data_module.prepare_data()
data_module.setup()


## Set up the model 

Here we use the previously trained `jsc-tiny` model in lab 1 as an example.

In [6]:
# If you stored your model checkpoint on Google Drive, remember to mount the drive to the current runtime in order to access it
# from google.colab import drive
# drive.mount('/content/drive')

In [7]:
# 📝️ change this CHECKPOINT_PATH to the one you trained in Lab1
CHECKPOINT_PATH = "/home/derek/mase/lab1_output/sweep-24-01-23-22-53-01/jsc-train-1/software/training_ckpts/best.ckpt"

# Returns a MaseModelInfo which is a dataclass that contains metadata about a particular model
model_info = get_model_info(model_name)

# This returns the nn.Module which represents the model/architecture
model = get_model(
    model_name,
    task="cls",
    dataset_info=data_module.dataset_info,
    pretrained=False)

# This loads pretrained params/weights into a model
model = load_model(load_name=CHECKPOINT_PATH, load_type="pl", model=model)
model

INFO     Loaded pytorch lightning checkpoint from /home/derek/mase/lab1_output/sweep-24-01-23-22-53-01/jsc-train-1/software/training_ckpts/best.ckpt


JSC_Tiny(
  (seq_blocks): Sequential(
    (0): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=16, out_features=5, bias=True)
    (3): ReLU(inplace=True)
  )
)

# Get a dummy data in
With the dataset module and model information, we can grab an input generator.

In [8]:
# get the input generator
input_generator = InputGenerator(
    data_module=data_module,
    model_info=model_info,
    task="cls",
    which_dataloader="train",
)

# a demonstration of how to feed an input value to the model
dummy_in = next(iter(input_generator))
print("dummy_in:", dummy_in)
output = model(**dummy_in)
print("output:", output)

dummy_in: {'x': tensor([[ 0.7269, -0.2892, -0.1055, -0.1972, -0.0528, -0.3965, -0.0318, -0.8361,
         -0.0318, -1.0361, -1.0466, -1.2401, -0.8093, -1.1989,  0.0878, -0.8917],
        [-0.2814, -0.5235, -1.1847, -0.8460, -1.0653, -0.6235,  0.3065,  0.7732,
          0.3065,  0.9158,  0.9166,  1.0545,  0.5065,  0.5138, -1.2489,  0.1694],
        [ 0.2963, -0.2281, -1.1774, -0.8438, -0.9882, -0.6182,  1.7951,  1.4778,
          1.7951,  0.9119,  0.3008, -0.0858,  1.2115,  0.8951, -1.2359, -0.6023],
        [-0.5348,  0.6949,  0.1801, -0.0708, -0.0127, -0.3261, -0.4939, -0.7763,
         -0.4939, -0.7057, -0.6650, -1.0233, -0.6948, -1.0035,  0.2401, -0.2647],
        [-1.3126,  0.8341,  1.8455,  2.1457,  2.3881,  2.8116,  0.5189,  0.6210,
          0.5189,  0.7785,  0.4358,  0.3688,  0.6317,  0.5732,  1.8627,  1.5199],
        [-0.5075,  0.3513, -0.3764, -0.6352, -0.2905, -0.4241,  0.0323,  0.2608,
          0.0323,  1.3362,  1.5906,  1.6988,  1.2465,  1.3509, -0.4428,  0.6517],
      

## Generate a MaseGraph
We have two forms of passes: transform passes and analysis passes, both of them would require the model to be transferred into a MaseGraph to allow manipulation.

In [9]:
# generate the mase graph and initialize node metadata
mg = MaseGraph(model=model)

## Running an Analysis pass
Analysis pass DOES NOT change the graph

The following analysis passes are essential to prepare the graph for other passes

In [10]:
# This initialises meta data on each torchfx node:
# node.meta["mase"] = MaseMetadata()
mg, _ = init_metadata_analysis_pass(mg, None)

# This assigns a bunch of MASE metadata based on the torchfx op type.
# A lot of rudimentary MASE metadata is added here and can be accessed via
# mase graph util functions.
mg, _ = add_common_metadata_analysis_pass(mg, {"dummy_in": dummy_in})

# Adds software metadata
mg, _ = add_software_metadata_analysis_pass(mg, None)

We will first run a simple graph analysis to understand the structure of the model.

In [11]:
# report graph is an analysis pass that shows you the detailed information in the graph
from chop.passes.graph import report_graph_analysis_pass
_ = report_graph_analysis_pass(mg)

graph():
    %x : [num_users=1] = placeholder[target=x]
    %seq_blocks_0 : [num_users=1] = call_module[target=seq_blocks.0](args = (%x,), kwargs = {})
    %seq_blocks_1 : [num_users=1] = call_module[target=seq_blocks.1](args = (%seq_blocks_0,), kwargs = {})
    %seq_blocks_2 : [num_users=1] = call_module[target=seq_blocks.2](args = (%seq_blocks_1,), kwargs = {})
    %seq_blocks_3 : [num_users=1] = call_module[target=seq_blocks.3](args = (%seq_blocks_2,), kwargs = {})
    return seq_blocks_3
Network overview:
{'placeholder': 1, 'get_attr': 0, 'call_function': 0, 'call_method': 0, 'call_module': 4, 'output': 1}
Layer types:
[BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True), ReLU(inplace=True), Linear(in_features=16, out_features=5, bias=True), ReLU(inplace=True)]


## Running another Analysis pass: Profile statistics

The pass `profile_statistics_analysis_pass` collects statistics of parameters and activations, and save them to node's metadata.

Here is a list of all the supported statistics. Refer to the `__init__` of statistic classes in `chop.passes.analysis.statistical_profiler.stat` to check the args each stat class takes.

This is a more complex analysis than the previous pass, and thus it would require you to pass in additional arguments for this pass.

### Example: the range of weights & input activations of nodes

Say we want to collect the tensor-wise min-max range of the 1st `torch.nn.Linear` nodes' weights & bias, and the channel-wise 97% quantile min-max of the 1st `torch.nn.Linear` nodes' input activations. We can do the following:

In [12]:

pass_args = {
    "by": "type",                                                            # collect statistics by node name
    "target_weight_nodes": ["linear"],                                       # collect weight statistics for linear layers
    "target_activation_nodes": ["relu"],                                     # collect activation statistics for relu layers
    "weight_statistics": {
        "variance_precise": {"device": "cpu", "dims": "all"},                # collect precise variance of the weight
    },
    "activation_statistics": {
        "range_quantile": {"device": "cpu", "dims": "all", "quantile": 0.97} # collect 97% quantile of the activation range
    },
    "input_generator": input_generator,                                      # the input generator for feeding data to the model
    "num_samples": 32,                                                       # feed 32 samples to the model
}

We can use the `report_node_meta_param_analysis_pass` to inspect the collected statistics.

In [13]:
mg, _ = profile_statistics_analysis_pass(mg, pass_args)
mg, _ = report_node_meta_param_analysis_pass(mg, {"which": ("software",)})

Profiling act statistics: 100%|██████████| 4/4 [00:00<00:00, 1258.13it/s]
INFO     Inspecting graph [add_common_meta_param_analysis_pass]
INFO     
+--------------+--------------+---------------------+--------------+-----------------------------------------------------------------------------------------+
| Node name    | Fx Node op   | Mase type           | Mase op      | Software Param                                                                          |
+==============+==============+=====================+==============+=========================================================================================+
| x            | placeholder  | placeholder         | placeholder  | {'results': {'data_out_0': {'stat': {}}}}                                               |
+--------------+--------------+---------------------+--------------+-----------------------------------------------------------------------------------------+
| seq_blocks_0 | call_module  | module              | bat

## Running a Transform pass: Quantisation

As its name suggests, the transform pass would modify the `MaseGraph`.
Similar to the previous analysis pass example, we would need to first declare the configuration for the pass.

In [14]:
pass_args = {
    "by": "type",
    "default": {"config": {"name": None}},
    "linear": {
        "config": {
            "name": "integer",
            # data
            "data_in_width": 8,
            "data_in_frac_width": 4,
            # weight
            "weight_width": 8,
            "weight_frac_width": 4,
            # bias
            "bias_width": 8,
            "bias_frac_width": 4,
        }
    },
}

We can then proceed to apply the transformation, in this case, we kept the original graph on purpose, so that we can print a `diff`.

In [15]:
from chop.passes.graph.transforms import (
    quantize_transform_pass,
    summarize_quantization_analysis_pass,
)
from chop.ir.graph.mase_graph import MaseGraph


ori_mg = MaseGraph(model=model)
ori_mg, _ = init_metadata_analysis_pass(ori_mg, None)
ori_mg, _ = add_common_metadata_analysis_pass(ori_mg, {"dummy_in": dummy_in})

mg, _ = quantize_transform_pass(mg, pass_args)
summarize_quantization_analysis_pass(ori_mg, mg, save_dir="quantize_summary")

INFO     Quantized graph histogram:
INFO     
| Original type   | OP           |   Total |   Changed |   Unchanged |
|-----------------+--------------+---------+-----------+-------------|
| BatchNorm1d     | batch_norm1d |       1 |         0 |           1 |
| Linear          | linear       |       1 |         1 |           0 |
| ReLU            | relu         |       2 |         0 |           2 |
| output          | output       |       1 |         0 |           1 |
| x               | placeholder  |       1 |         0 |           1 |


In [16]:
from chop.passes.graph.analysis import (
    travese_graph_analysis_pass,
)

travese_graph_analysis_pass(ori_mg)

INFO     
| Torch Name   | MASE Type           | Mase Op      | Common Params                                                                                                                                                                                                                                                               | Software Params   | Hardware Params   |
|--------------+---------------------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+-------------------|
| x            | placeholder         | placeholder  | {'mase_type': 'placeholder', 'mase_op': 'placeholder', 'args': {}, 'results': {'data_out_0': {'type': 'float', 'precision': [32], 'shape': [8, 16], 'torch_dtype': torch.float32, 'value': tensor([[ 0.7269, -0.

,Torch Name,MASE Type,Mase Op,Common Params,Software Params,Hardware Params
0,x,placeholder,placeholder,"{'mase_type': 'placeholder', 'mase_op': 'place...",{},{}
1,seq_blocks_0,module,batch_norm1d,"{'mase_type': 'module', 'mase_op': 'batch_norm...",{},{}
2,seq_blocks_1,module_related_func,relu,"{'mase_type': 'module_related_func', 'mase_op'...",{},{}
3,seq_blocks_2,module_related_func,linear,"{'mase_type': 'module_related_func', 'mase_op'...",{},{}
4,seq_blocks_3,module_related_func,relu,"{'mase_type': 'module_related_func', 'mase_op'...",{},{}
5,output,output,output,"{'mase_type': 'output', 'mase_op': 'output', '...",{},{}


In [17]:
travese_graph_analysis_pass(mg)

INFO     
| Torch Name   | MASE Type           | Mase Op      | Common Params                                                                                                                                                                                                                                                               | Software Params                                                                                                                                                                                                                                                                                                            | Hardware Params   |
|--------------+---------------------+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------

,Torch Name,MASE Type,Mase Op,Common Params,Software Params,Hardware Params
0,x,placeholder,placeholder,"{'mase_type': 'placeholder', 'mase_op': 'place...",{'results': {'data_out_0': {'stat': {}}}},{}
1,seq_blocks_0,module,batch_norm1d,"{'mase_type': 'module', 'mase_op': 'batch_norm...","{'args': {'data_in_0': {'stat': {}}, 'weight':...",{}
2,seq_blocks_1,module_related_func,relu,"{'mase_type': 'module_related_func', 'mase_op'...",{'args': {'data_in_0': {'stat': {'range_quanti...,{}
3,seq_blocks_2,module_related_func,linear,"{'mase_type': 'module_related_func', 'mase_op'...","{'args': {'data_in_0': {'stat': {}}, 'weight':...",{}
4,seq_blocks_3,module_related_func,relu,"{'mase_type': 'module_related_func', 'mase_op'...",{'args': {'data_in_0': {'stat': {'range_quanti...,{}
5,output,output,output,"{'mase_type': 'output', 'mase_op': 'output', '...",{'args': {'data_in_0': {'stat': {}}}},{}




# Exercises:

We have now seen how to:
1. Set up a dataset
2. Set up a model
3. Generate a `MaseGraph` from the model
4. Run Analysis and Transform passes on the `MaseGraph`

Now consider the following problems:

1. Explain the functionality of `report_graph_analysis_pass` and its printed jargons such as `placeholder`, `get_attr` ... You might find the doc of [torch.fx](https://pytorch.org/docs/stable/fx.html) useful.

2. What are the functionalities of `profile_statistics_analysis_pass` and `report_node_meta_param_analysis_pass` respectively?

## MASE OPs and MASE Types

MASE is designed to be a very high-level intermediate representation (IR), this is very different from the classic [LLVM IR](https://llvm.org/docs/LangRef.html) that you might be familiar with.

The following MASE Types are available:
(Note from Aaron: do we have a page somewhere that have summarized this?)


## A deeper dive into the quantisation transform

3. Explain why only 1 OP is changed after the `quantize_transform_pass` .

4. Write some code to traverse both `mg` and `ori_mg`, check and comment on the nodes in these two graphs. You might find the source code for the implementation of `summarize_quantization_analysis_pass` useful.

5. Perform the same quantisation flow to the bigger JSC network that you have trained in lab1. You must be aware that now the `pass_args` for your custom network might be different if you have used more than the `Linear` layer in your network.

6. Write code to show and verify that the weights of these layers are indeed quantised. You might need to go through the source code of the implementation of the quantisation pass and also the implementation of the [Quantized Layers](../../machop/chop/passes/transforms/quantize/quantized_modules/linear.py) .

## The command line interface

The same flow can also be executed on the command line throw the `transform` action.

```bash
# make sure you have the same printout
pwd
# it should show
# your_dir/mase-tools/machop

# enter the following command
./ch transform --config configs/examples/jsc_toy_by_type.toml --task cls --cpu=0
```
7. Load your own pre-trained JSC network, and perform perform the quantisation using the command line interface.

## \[Optional] Write your own pass

Many examples of existing passes are in the [source code](../..//machop/chop/passes/__init__.py), the [test files](../../machop/test/passes) for these passes also contain useful information on helping you to understand how these passes are used.

Implement a pass to count the number of FLOPs (floating-point operations) and BitOPs (bit-wise operations).